In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# file_path = os.path.join('D:', 'MINI PROJECT', 'Data', 'train.csv')
file_path = '/content/drive/My Drive/toxic-comment/train.csv'

df = pd.read_csv(file_path)



In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


  preprocess


In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
x=df['comment_text']
y=df[df.columns[2: ]].values

In [ ]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [ ]:
x

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [ ]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
Max_features =  200000 #num of words

In [ ]:
vectorizer=TextVectorization(max_tokens=Max_features,
                             output_sequence_length=1800,
                             output_mode='int')

In [ ]:
vectorizer.adapt(x.values)

In [ ]:
vectorizer('Hello world, life is great')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([288, 263, 306,   9, 275])>

In [ ]:
# vectorizer.get_vocabulary()

In [ ]:
vectorized_text=vectorizer(x.values)

In [ ]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)

In [ ]:
batch_x,batch_y=dataset.as_numpy_iterator().next()

In [ ]:
train=dataset.take(int(len(dataset)*.7))
val=dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [ ]:
train.as_numpy_iterator().next()

(array([[   8,  210,   10, ...,    0,    0,    0],
        [   2,  403,  378, ...,    0,    0,    0],
        [ 797,  136, 2591, ...,    0,    0,    0],
        ...,
        [   1,    7,  580, ...,    0,    0,    0],
        [   2,    1, 2477, ...,    0,    0,    0],
        [ 143,  804,   36, ...,    0,    0,    0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 1]]))

DEEP LEARNING NETWORK

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding

In [ ]:
model=Sequential()

model.add(Embedding(Max_features+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
history=model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 779s 110ms/step - loss: 0.0614 - val_loss: 0.0447


In [ ]:
history.history

{'loss': [0.061428502202034], 'val_loss': [0.04474626109004021]}

In [ ]:
# from matplotlib import pyplot as plt

In [ ]:
# plt.figure(figsize=(8,5))
# pd.DataFrame(history.history).plot()
# plt.show()

prediction

In [ ]:
batch=test.as_numpy_iterator().next()

In [ ]:
input_text=vectorizer("You are good")

In [ ]:
model.predict(np.array([input_text]))

1/1 [==============================] - 1s 1s/step


array([[0.19818541, 0.00205498, 0.04023634, 0.01262689, 0.050951  ,
        0.0128284 ]], dtype=float32)

In [ ]:
batch = test.as_numpy_iterator().next()


In [ ]:
batch_x,batch_y = test.as_numpy_iterator().next()

In [ ]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
(model.predict(batch_x)>0.5).astype(int)

1/1 [==============================] - 0s 61ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
res=model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 52ms/step


Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [ ]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    x_true,y_true=batch
    yhat=model.predict(x_true)
    y_true=y_true.flatten()
    yhat=yhat.flatten()

    pre.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    acc.update_state(y_true,yhat)


1/1 [==============================] - 0s 53ms/step


In [ ]:
print("precision : ",pre.result().numpy())

precision :  0.8439263


In [ ]:
print("Recall : ",re.result().numpy())

Recall :  0.6576788


In [ ]:
print("Accuracy : ",acc.result().numpy())

Accuracy :  0.45436308


Webscraping

In [ ]:
# !pip install beautifulsoup4

In [ ]:
# !pip install requests

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# # Send a GET request to the Wikipedia page
# url = input("Enter URL:")
# response = requests.get(url)

# if response.status_code == 200:
#     # Parse the HTML content
#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Find all paragraphs in the main content
#     paragraphs = soup.find_all("p")

#     if paragraphs:
#         # Extract and join the text within the paragraphs
#         text = "\n".join(paragraph.get_text() for paragraph in paragraphs)

#         # Save the extracted text to a file
#         with open("extracted_text.txt", "w", encoding="utf-8") as file:
#             file.write(text)

#         print("Text extracted and saved to 'extracted_text.txt'")
#     else:
#         print("No paragraphs found in the main content.")
# else:
#     print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [ ]:
# Read the contents of the file and store it as input_text
# with open("extracted_text.txt", "r", encoding="utf-8") as file:
#     input_text = file.read()

# Now, input_text contains the text from the file
# print(input_text)
from tensorflow.keras.models import load_model
model_path = '/content/drive/My Drive/toxic_comment_classification_model10.h5'
model = load_model(model_path)


input_texttt=input("Enter Text: ")
input_text=vectorizer(input_texttt)
prediction = model.predict(np.array([input_text]))
column_names = df.columns[2:]

prediction_df = pd.DataFrame(prediction, columns=column_names)

print(prediction_df)


Enter Text: Anirudh Ravichander (born 16 October 1990), also credited mononymously as Anirudh, is an Indian music composer, and singer who works primarily in Tamil Cinema. He has also composed music for few Hindi and Telugu films. He has won two Filmfare Awards South, nine SIIMA Awards, six Edison Awards and five Vijay Awards.[2] His debut song "Why This Kolaveri Di", composed for the 2012 film 3, went viral across the globe and has achieved over 400 million views on YouTube.[3] A.R. Murugadoss signed him to compose music for Kaththi (2014) starring Vijay,[4] which included the viral hit "Selfie Pulla".[5] The soundtrack for the film became Anirudh's highest profile soundtrack until he was signed to compose music for Rajinikanth's Petta in 2019.[5] In 2016, he signed a record deal with Sony Music, which publishes his independent albums and live concerts.[6] In the same year, he featured with Diplo on the remix of Major Lazer's hit single "Cold Water".[7] Anirudh Ravichander is the son 

In [ ]:
import pickle

In [ ]:
filename='model10.pkl'

In [ ]:
with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
model.save('toxic_comment_classification_model10')


In [ ]:
model.save('toxic_comment_classification_model10.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loaded_model = tf.keras.models.load_model('toxic_comment_classification_model10')


In [ ]:
# Save the vectorizer configuration
import json
config = vectorizer.get_config()
with open('vectorizer_config10.json', 'w') as f:
    json.dump(config, f)


In [ ]:
# Load the vectorizer configuration
with open('vectorizer_config10.json', 'r') as f:
    config = json.load(f)
vectorizer = tf.keras.layers.TextVectorization.from_config(config)


In [ ]:
with open('training_history10.json', 'w') as f:
    json.dump(history.history, f)


In [ ]:
with open('training_history10.json', 'r') as f:
    loaded_history = json.load(f)


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Move your saved files to the Google Drive directory
!mv model10.pkl /content/drive/My\ Drive/
!mv toxic_comment_classification_model10 /content/drive/My\ Drive/
!mv toxic_comment_classification_model10.h5 /content/drive/My\ Drive/
!mv vectorizer_config10.json /content/drive/My\ Drive/
!mv training_history10.json /content/drive/My\ Drive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mv: cannot stat 'model10.pkl': No such file or directory
mv: cannot stat 'toxic_comment_classification_model10': No such file or directory
mv: cannot stat 'toxic_comment_classification_model10.h5': No such file or directory
mv: cannot stat 'vectorizer_config10.json': No such file or directory
